In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import tensorflow

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

#config = tf.compat.v1.ConfigProto(device_count={'GPU': 2}, allow_soft_placement=True, log_device_placement=True)
config = tensorflow.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.8
tensorflow.compat.v1.Session(config=config)
tensorflow.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



'2.4.0-dev20200815'

In [2]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

from tensorflow.keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

In [3]:
# SPECIFY WHICH EXPERIMENT TO LOAD BY INDEX IN EXPERIMENT-LOG.JSON
ind = 6

In [6]:
from tensorflow.keras.optimizers import Adam

with open("logs/unvalidated-experiment-log.json") as f:
  experiments = json.load(f)
experiment = experiments['experiments'][ind]
experiment_benchmark = experiments['experiments'][ind]['benchmark']

experiment_benchmark['name'] = experiment_benchmark['benchmark_name']
experiment_benchmark['dataset_folder'] = Path(experiment_benchmark['dataset_folder'])

experiment_benchmark.pop('benchmark_name', None)
experiment_benchmark.pop('num_samples_train', None)
experiment_benchmark.pop('num_samples_validation', None)
experiment_benchmark.pop('num_samples_test', None)

if experiment_benchmark['optimizer'] == 'Adam':
    lr = experiment_benchmark['learning_rate']
    experiment_benchmark['optimizer'] = Adam(learning_rate=lr)
    experiment_benchmark.pop('learning_rate', None)
else:
    raise NotImplementedError()

posw = tensorflow.constant(experiment_benchmark['positive_weights'], dtype=tensorflow.float32)
negw = tensorflow.constant(experiment_benchmark['negative_weights'], dtype=tensorflow.float32)

experiment_benchmark.pop('positive_weights', None)
experiment_benchmark.pop('negative_weights', None)

experiment_benchmark['test_labels'] = "test.csv"

benchmark = Benchmark(**experiment_benchmark)
testgen = benchmark.testgen
traingen = benchmark.traingen
valgen = benchmark.valgen

print(experiment_benchmark['name'])

Chexpert_BCE_E5_B52_C0_N5_D256_DS9505_LR4


In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from src.metrics.metrics import SingleClassMetric, NaNWrapper

path = os.path.join("models/", experiment['name'], experiment['filename'])
reconstructed_model = models.load_model(path, compile=False)

if experiment_benchmark['loss'] == "weighted_binary_crossentropy":
    loss = WeightedBinaryCrossentropy(posw, negw)
elif experiment_benchmark['loss'] == "binary_crossentropy":
    loss = BinaryCrossentropy()
else:
    raise NotImplementedError()

reconstructed_model.compile(optimizer=Adam(), loss=loss, metrics=[Precision(), Recall()])

In [8]:
# testgen.on_epoch_end()
print(len(testgen))

234


In [9]:
thresholds = [0.3, 0.4, 0.5]

In [ ]:
compute_class_weight(traingen)

In [10]:
groundtruth_label = testgen.get_labels_nonan()
predictions = reconstructed_model.predict(testgen, steps=len(testgen), verbose=1)

for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (predictions >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()
    


234/234 [==============================] - 11s 47ms/step
current threshold:  0.3
TF Precision:  tf.Tensor(0.33027524, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.12286689, shape=(), dtype=float32)
current threshold:  0.4
TF Precision:  tf.Tensor(0.35555556, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.10921501, shape=(), dtype=float32)
current threshold:  0.5
TF Precision:  tf.Tensor(0.3561644, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.088737205, shape=(), dtype=float32)


In [ ]:
predictions
groundtruth_label = testgen.get_labels()

In [ ]:
report = classification_report(groundtruth_label, y_pred, target_names=benchmark.label_columns)
print('sklearn report: ', report)

In [ ]:
eval_res = reconstructed_model.evaluate(
            x=testgen, steps=len(testgen), verbose=1)

In [ ]:
for x in predictions:
    print(x)

In [11]:
from numpy import genfromtxt
my_data = genfromtxt('models/DenseNet121_Chexpert_BCE_E5_B52_C0_N5_D256_DS9505_LR4/predictions_probs.csv', delimiter=';')

In [12]:
predictions_bool_mod = (my_data >= 0.5)
y_pred_mod = np.array(predictions_bool_mod, dtype=int)

report = classification_report(groundtruth_label, y_pred_mod, target_names=benchmark.label_columns)
print('sklearn report: ', report)

sklearn report:                    precision    recall  f1-score   support

    Cardiomegaly       0.25      0.01      0.03        68
           Edema       0.12      0.09      0.10        45
   Consolidation       0.00      0.00      0.00        33
     Atelectasis       0.67      0.03      0.05        80
Pleural Effusion       0.35      0.27      0.30        67

       micro avg       0.26      0.09      0.13       293
       macro avg       0.28      0.08      0.10       293
    weighted avg       0.34      0.09      0.10       293
     samples avg       0.09      0.05      0.06       293



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
print(np.sum(predictions_bool))
np.sum(predictions_bool_mod)

73


96

I think something doesnt work with the masked loss function, I get really low values here. Fix.